In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from common import *
import qgrid

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Balance

In [ ]:
from portfolio.balance import BalanceType
b = Balance()
print("Currencies",  b.currencies)
print("Entries", b.to_dict())
b.add_currency(c.ETH)
print("Currencies",  b.currencies)
b.update(c.ETH, delta_free=1.0, delta_used=0.0)
print(c.ETH, b.get(c.ETH)[BalanceType.FREE])

### Exchange

In [ ]:
# Public informaiton
exchanges = [c.PAPER, c.BINANCE, c.GDAX]#, c.POLONIEX]
a = Asset(c.ETH, c.BTC)
for ex in exchanges:
    print("Exchange", ex)
    exchange = load_exchange(ex)
    print(exchange.timeframes)
    exchange.get_markets()
    exchange.fetch_ohlcv(a, Timeframe.ONE_MIN.value['id'])
    exchange.fetch_order_book(a)
    exchange.fetch_public_trades(a)
    exchange.fetch_ticker(a)

In [ ]:
# Account Information
exchanges = [c.PAPER, c.BINANCE, c.GDAX]
a = Asset(c.ETH, c.BTC)
for ex in exchanges:
    print("Exchange", ex)
    exchange = load_exchange(ex)
    print(exchange.fetch_balance().to_dict())
    print(exchange.fetch_orders(a))
    print(exchange.fetch_open_orders(a))
    print(exchange.fetch_closed_orders(a))

In [ ]:
exchange = load_exchange(c.BINANCE)
a = Asset(c.BTS, c.BTC)

# Market BUY ... CAREFUL THIS WILL ACTUALLY BUY
print("Exchange", exchange)
#order_dict = exchange.create_market_buy_order(a, 1)
balance = exchange.fetch_balance()
print(a.base, balance.get(a.base), a.quote, balance.get(a.quote))
print(order_dict)

In [ ]:
# Check Order
balance = exchange.fetch_balance()
print(balance.to_dict())
print(a.base, balance.get(a.base), a.quote, balance.get(a.quote))
found_order = exchange.fetch_order(order_dict['id'], a.symbol)
print("Placed order", found_order) 

In [ ]:
# Sell all remaining quantity
print(a.base)
balance = exchange.fetch_balance()
print(a.base, balance.get(a.base)[BalanceType.TOTAL])
order = exchange.create_market_sell_order(a, balance.get(a.base)[BalanceType.TOTAL])
print(order)

In [ ]:
print(exchange.fetch_balance()[a.base], exchange.fetch_balance()[a.quote])

### Data

In [ ]:
base = c.LTC
quote = c.USDT
exchange = load_exchange(c.BINANCE)
asset = Asset(base, quote)
assets = [ Asset(coin, quote) for coin in [c.BTC, c.ETH, c.LTC] ]
period = '1m' 
start = datetime.datetime.utcnow() - datetime.timedelta(hours=2)
end = datetime.datetime.utcnow() - datetime.timedelta(hours=0)

In [ ]:
# Single Coin
df = ohlcv.fetch_and_save_ohlcv_data(exchange, asset, period, start, end)
df.head()

In [ ]:
# Load from File
fpath = ohlcv.get_price_data_fpath(asset, exchange.id, period)
df = ohlcv.load_chart_data_from_file(fpath)

In [ ]:
# Multiple Coins
ohlcv.download_chart_data(exchange, assets, period, start, end)
df = ohlcv.load_multiple_assets(exchange.id, assets, period, start)
df.head()

### Data Feed

In [ ]:
# CSV Feed
feed_fpath = ohlcv.get_price_data_fpath(asset, exchange.id, period='1m')
feed = CSVDataFeed(feed_fpath)
feed.initialize()

In [ ]:
# Grab 1 row at a time
for i in range(3):
    data = feed.next()
    print(data['time_utc'], data['close'])

In [ ]:
# Access all rows in history
feed.history().head()

In [ ]:
# Exchange Live Feed
start = datetime.datetime.utcnow() - datetime.timedelta(hours=1)
end = None
feed_fpath = ohlcv.get_price_data_fpath(asset, exchange.id, period='1m')

In [ ]:
feed = ExchangeDataFeed(exchange, [assets[0]], period, feed_fpath, start, end)
feed.initialize()

In [ ]:
# Access History
feed.history(t_minus=3).tail()

In [ ]:
# Grab 1 row at a time (don't refresh data)
for i in range(3):
    data = feed.next(refresh=False)
    print(data['time_utc'], data['close'])

In [ ]:
# Refresh data before next query
data = feed.next(refresh=True)
print(data['time_utc'], data['close'])

In [ ]:
# Example Using the Feed

def my_strategy(row, history, exchange):
    print("Time:", row['time_utc'], "Price:", row['close'])
    if row['close'] > .024:
        print("Buying coin")
        #res = exchange.limit_buy(quantity=1, price=5)
    else:
        print("Selling that stupid coin")
    return {
        'time': row['time_utc'],
        'close': row['close'],
        'action': 'buy',
        'quantity': 1,
        'price': 5,
        'exchange': exchange.id
    }

def backtest(feed, strategy, exchange):
    results = []
    row = feed.next()
    while row is not None:
        output = strategy(row, feed, exchange)
        row = feed.next(refresh=True)
        results.append(output)
    return results

In [ ]:
res = backtest(feed, my_strategy, exchange)

### Order

In [ ]:
asset = Asset(c.LTC, c.USDT)
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=250., 
    quantity=1, 
    order_type=OrderType.LIMIT_BUY
)
order

### Position

In [ ]:
asset = Asset(c.LTC, c.BTC)
pos = Position(asset, quantity=1, cost_price=250.0)
print("Asset", pos.asset.to_dict())
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

pos.update(1, 200)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

pos.update(-1, 200)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

# Go short
pos.update(-2, 150)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

# Close the short for a LOSS and go long (price went up, we had to pay $300 to cover our short)
# then we bought another share because we're newbs
pos.update(2, 300)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

In [ ]:
"""
Assume that an investor made the following consecutive fund purchases in a taxable account: 1,500 shares at $20, 1,000 shares at $10 and 1,250 shares at $8. The investor’s average cost basis is calculated by dividing $50,000/3,750 shares. The average cost is $13.33.

Suppose the investor then sells 1,000 shares of the fund at $19.

Gain/loss using average cost basis: ($19 - $13.33) x 1,000 shares = $5,667
"""

asset = Asset(c.LTC, c.BTC)
pos = Position(asset, quantity=1500, cost_price=20.0)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)
pos.update(txn_quantity=1000, txn_price=10)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)
pos.update(txn_quantity=1250, txn_price=8)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

last_cost = pos.cost_price
pos.update(txn_quantity=-1000, txn_price=19)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)
print("Profit", (19 - last_cost) * 1000)

### PerformanceTracker

In [ ]:
asset = Asset(c.BTC, c.USD)
perf = PerformanceTracker(starting_cash=5000, timeframe=Timeframe.ONE_MIN, store=None)
perf.to_dict()

In [ ]:
# Buy 1 BTC for $1000
pos1 = Position(asset, quantity=1, cost_price=1000.0)
positions = [pos1]
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=4000.0,
    positions=positions
)
perf.to_dict()

In [ ]:
"""
Value of BTC increased $100
Position Return
    Return = .10
    PnL = $100
Cumulative Return
    Return = $100 / $5000 = .02
    PnL = $100
"""
pos1.latest_price = 1100
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=4000.0,
    positions=positions
)
perf.to_dict()

In [ ]:
"""
Value of BTC increased again $400
Position Return
    Return = .5
    PnL = $500
Cumulative Return
    Return = $500 / $5000 = .1
    PnL = $500
"""
pos1.latest_price = 1500
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=4000.0,
    positions=positions
)
perf.to_dict()

In [ ]:
# Buy 1 ETH for $500
asset = Asset(c.ETH, c.USD)
pos2 = Position(asset, quantity=1, cost_price=500.0)
positions.append(pos2)
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=3500.0,
    positions=positions
)
perf.to_dict()

In [ ]:
"""
Value of ETH decreased $100
Position Return
    Return = -100/500
    PnL = -100
Cumulative Return
    Return = 400 / 5000 = .08
    PnL = 400
"""
pos2.latest_price = 400
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=3500.0,
    positions=positions
)
perf.to_dict()

### Portfolio

In [ ]:
starting_cash = 5000
exchange = load_exchange(c.PAPER)
perf = PerformanceTracker(starting_cash, Timeframe.ONE_MIN, store=None)
portfolio = Portfolio(starting_cash, perf)

In [ ]:
# Buy 1 BTC for $1000
asset = Asset(c.BTC, c.USD)
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=1000., 
    quantity=1, 
    order_type=OrderType.LIMIT_BUY
)
order

In [ ]:
portfolio.update(filled_orders=[order])
portfolio.to_dict()

In [ ]:
# Buy more BTC, price has risen $500
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=1500., 
    quantity=1, 
    order_type=OrderType.LIMIT_BUY
)
portfolio.update(filled_orders=[order])
portfolio.to_dict()

In [ ]:
# Sell all BTC at profit
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=1500., 
    quantity=2, 
    order_type=OrderType.LIMIT_SELL
)
portfolio.update(filled_orders=[order])
portfolio.to_dict()

In [ ]:
portfolio.perf.to_dict()

### Strategy

In [ ]:
# https://www.backtrader.com/docu/quickstart/quickstart.html
# https://enigmampc.github.io/catalyst/beginner-tutorial.html#basics

class Strategy():
    def __init__(self, params):
        self.params = params
        self.initialize()
    
    def initialize(self):
        self.ctx.symbol = 'BTC/USD'
    
    def handle_data(self, data):
        # Handle next iteration of data from feed
        pass
    
class TestStrategy():
    def __init__(self):
        pass
    
    def next(self, data):
        pass

### Plotting

In [ ]:
utils.charts.plot_range(feed.history(), start, end, 'close')

### Runner

In [ ]:
@unique
class TradeMode(Enum):
    BACKTEST = 0
    SIMULATE = 1
    LIVE = 2


class Punisher():
    def __init__(self, exchange, feed, strategy, record):
        self.exchange = exchange
        self.feed = feed
        self.strategy = strategy
        self.record = record
        
    def backtest(self):
        results = []
        row = self.feed.next()
        while row is not None:
            print("Timestep", row['time_utc'], "Price", row['close'])
            row = self.feed.next()
            output = self.strategy(row, self.exchange, self.feed)
            results.append(output)
        return results
    
    def simulate(self):
        while True:
            row = self.feed.next()
            if row is not None:
                output = self.strategy(row, self.exchange, self.feed)
                self.record['test'].append(output)
            time.sleep(2)

    def live(self):
        while True:
            row = self.feed.next()
            if row is not None:
                output = self.strategy(row, self.exchange, self.feed)
                self.record['live'].append(output)
            time.sleep(2)
    
    def punish(self, mode=False):
        if mode == TradeMode.BACKTEST:
            print("Backtesting ...")
            self.backtest()
        elif mode == TradeMode.SIMULATE:
            print("Simulating orders ...")
            self.simulate()
        elif mode == TradeMode.LIVE:
            print("LIVE TRADING! CAREFUL!")
            self.live()

# PaperExchangeDataProvider

In [ ]:

base = c.LTC
quote = c.USDT
exchange = load_exchange(c.BINANCE)
asset = Asset(base, quote)
assets = [ Asset(coin, quote) for coin in [c.BTC, c.ETH, c.LTC] ]
period = '1m' 
start = datetime.datetime.utcnow() - datetime.timedelta(hours=2)
end = datetime.datetime.utcnow() - datetime.timedelta(hours=0)

feed_fpath = ohlcv.get_price_data_fpath(asset, exchange.id, period='1m')
feed = CSVDataFeed(feed_fpath)

data_provider = PaperExchangeDataProvider(feed)
config = {
    'data_provider': data_provider,
    'balance': Balance()
}

exchange = load_exchange("paper", config)


In [ ]:
# Testing Paper Exchange Data Provider
print(exchange.get_markets())
print(exchange.fetch_ohlcv(asset, '1m'))
print(exchange.fetch_order_book(asset))
print("TICKER")
print(exchange.fetch_ticker(asset))
print(exchange.timeframes)
print(exchange.fetch_public_trades(asset))

In [5]:
context = Context.from_config()
punisher.punish(context, SimpleStrategy())

Loading feed: /Users/cfortuner/Dropbox/punisher/default_feed.csv
Downloading: ETH/BTC
Downloaded rows: 500
Punishing ...
Timestep 2018-01-09 17:54:00 Price 0.08082
2018-01-10T04:34:56.234118, Epoch 1
2018-01-10T04:34:56.234374, open                    0.08076
high                    0.08095
low                    0.080759
close                  0.080824
volume                  178.429
time_utc    2018-01-09 17:55:00
Name: 1515520500, dtype: object
Timestep 2018-01-09 17:55:00 Price 0.080824
2018-01-10T04:34:57.239972, Epoch 2
2018-01-10T04:34:57.240175, open                   0.080805
high                   0.080926
low                    0.080757
close                  0.080802
volume                  184.129
time_utc    2018-01-09 17:56:00
Name: 1515520560, dtype: object
Timestep 2018-01-09 17:56:00 Price 0.080802
2018-01-10T04:34:58.247235, Epoch 3
2018-01-10T04:34:58.247529, open                   0.080802
high                     0.0809
low                    0.080731
close       

Timestep 2018-01-09 18:19:00 Price 0.081044
2018-01-10T04:35:21.416258, Epoch 26
2018-01-10T04:35:21.416331, open                   0.080953
high                   0.081044
low                    0.080856
close                  0.080949
volume                  206.513
time_utc    2018-01-09 18:20:00
Name: 1515522000, dtype: object
Timestep 2018-01-09 18:20:00 Price 0.080949
2018-01-10T04:35:22.426276, Epoch 27
2018-01-10T04:35:22.426668, open                   0.080949
high                   0.081007
low                     0.08087
close                  0.080924
volume                  172.785
time_utc    2018-01-09 18:21:00
Name: 1515522060, dtype: object
Timestep 2018-01-09 18:21:00 Price 0.080924
2018-01-10T04:35:23.437564, Epoch 28
2018-01-10T04:35:23.437828, open                   0.080996
high                      0.081
low                      0.0809
close                  0.080987
volume                  184.279
time_utc    2018-01-09 18:22:00
Name: 1515522120, dtype: object
T

Timestep 2018-01-09 18:44:00 Price 0.08142
2018-01-10T04:35:46.617743, Epoch 51
2018-01-10T04:35:46.617809, open                    0.08151
high                   0.081556
low                    0.081414
close                  0.081521
volume                  165.235
time_utc    2018-01-09 18:45:00
Name: 1515523500, dtype: object
Timestep 2018-01-09 18:45:00 Price 0.081521
2018-01-10T04:35:47.624259, Epoch 52
2018-01-10T04:35:47.625229, open                   0.081521
high                   0.081551
low                     0.08135
close                  0.081418
volume                  164.895
time_utc    2018-01-09 18:46:00
Name: 1515523560, dtype: object
Timestep 2018-01-09 18:46:00 Price 0.081418
2018-01-10T04:35:48.633717, Epoch 53
2018-01-10T04:35:48.633780, open                   0.081528
high                   0.081541
low                    0.081366
close                  0.081532
volume                  203.412
time_utc    2018-01-09 18:47:00
Name: 1515523620, dtype: object
Ti

Timestep 2018-01-09 19:09:00 Price 0.081709
2018-01-10T04:36:11.812113, Epoch 76
2018-01-10T04:36:11.812177, open                   0.081747
high                   0.081754
low                    0.081667
close                  0.081676
volume                  182.781
time_utc    2018-01-09 19:10:00
Name: 1515525000, dtype: object
Timestep 2018-01-09 19:10:00 Price 0.081676
2018-01-10T04:36:12.818239, Epoch 77
2018-01-10T04:36:12.818307, open                   0.081676
high                   0.081754
low                     0.08159
close                   0.08159
volume                  157.554
time_utc    2018-01-09 19:11:00
Name: 1515525060, dtype: object
Timestep 2018-01-09 19:11:00 Price 0.08159
2018-01-10T04:36:13.825472, Epoch 78
2018-01-10T04:36:13.825727, open                   0.081677
high                   0.081716
low                     0.08159
close                   0.08165
volume                  285.041
time_utc    2018-01-09 19:12:00
Name: 1515525120, dtype: object
Ti

Timestep 2018-01-09 19:34:00 Price 0.081673
2018-01-10T04:36:37.009485, Epoch 101
2018-01-10T04:36:37.009707, open                   0.081737
high                   0.081759
low                    0.081662
close                   0.08167
volume                  157.856
time_utc    2018-01-09 19:35:00
Name: 1515526500, dtype: object
Timestep 2018-01-09 19:35:00 Price 0.08167
2018-01-10T04:36:38.019455, Epoch 102
2018-01-10T04:36:38.019521, open                   0.081671
high                   0.081822
low                    0.081671
close                  0.081746
volume                  153.536
time_utc    2018-01-09 19:36:00
Name: 1515526560, dtype: object
Timestep 2018-01-09 19:36:00 Price 0.081746
2018-01-10T04:36:39.029531, Epoch 103
2018-01-10T04:36:39.029643, open                   0.081746
high                    0.08188
low                    0.081746
close                  0.081852
volume                  221.889
time_utc    2018-01-09 19:37:00
Name: 1515526620, dtype: object

...

Timestep 2018-01-10 02:39:00 Price 0.091776
2018-01-10T04:43:45.166110, Epoch 526
2018-01-10T04:43:45.166136, open                   0.091785
high                      0.092
low                    0.091776
close                  0.091911
volume                  210.275
time_utc    2018-01-10 02:40:00
Name: 1515552000, dtype: object
Timestep 2018-01-10 02:40:00 Price 0.091911
2018-01-10T04:43:46.175167, Epoch 527
2018-01-10T04:43:46.175200, open                   0.091999
high                   0.092046
low                    0.091911
close                  0.092046
volume                  404.818
time_utc    2018-01-10 02:41:00
Name: 1515552060, dtype: object
Timestep 2018-01-10 02:41:00 Price 0.092046
2018-01-10T04:43:47.184114, Epoch 528
2018-01-10T04:43:47.184139, open                   0.092002
high                     0.0925
low                    0.092001
close                  0.092054
volume                  503.831
time_utc    2018-01-10 02:42:00
Name: 1515552120, dtype: objec

Timestep 2018-01-10 03:04:00 Price 0.092599
2018-01-10T04:44:10.377867, Epoch 551
2018-01-10T04:44:10.377897, open                   0.092599
high                    0.09276
low                    0.092554
close                  0.092724
volume                  272.405
time_utc    2018-01-10 03:05:00
Name: 1515553500, dtype: object
Timestep 2018-01-10 03:05:00 Price 0.092724
2018-01-10T04:44:11.386434, Epoch 552
2018-01-10T04:44:11.386466, open                   0.092724
high                   0.093203
low                    0.092704
close                   0.09311
volume                  534.099
time_utc    2018-01-10 03:06:00
Name: 1515553560, dtype: object
Timestep 2018-01-10 03:06:00 Price 0.09311
2018-01-10T04:44:12.396319, Epoch 553
2018-01-10T04:44:12.396351, open                    0.09311
high                     0.0935
low                     0.09311
close                   0.09342
volume                    652.2
time_utc    2018-01-10 03:07:00
Name: 1515553620, dtype: object

Timestep 2018-01-10 03:29:00 Price 0.094741
2018-01-10T04:44:35.593164, Epoch 576
2018-01-10T04:44:35.593192, open                   0.094726
high                     0.0951
low                    0.094726
close                  0.094946
volume                  510.714
time_utc    2018-01-10 03:30:00
Name: 1515555000, dtype: object
Timestep 2018-01-10 03:30:00 Price 0.094946
2018-01-10T04:44:36.602895, Epoch 577
2018-01-10T04:44:36.602953, open                   0.095043
high                    0.09539
low                       0.095
close                   0.09539
volume                  484.306
time_utc    2018-01-10 03:31:00
Name: 1515555060, dtype: object
Timestep 2018-01-10 03:31:00 Price 0.09539
2018-01-10T04:44:37.610744, Epoch 578
2018-01-10T04:44:37.610777, open                   0.095305
high                   0.095498
low                    0.095304
close                  0.095474
volume                  453.191
time_utc    2018-01-10 03:32:00
Name: 1515555120, dtype: object

Timestep 2018-01-10 03:54:00 Price 0.098106
2018-01-10T04:45:00.805153, Epoch 601
2018-01-10T04:45:00.805182, open                   0.098106
high                   0.098211
low                    0.097107
close                   0.09741
volume                  1178.48
time_utc    2018-01-10 03:55:00
Name: 1515556500, dtype: object
Timestep 2018-01-10 03:55:00 Price 0.09741
2018-01-10T04:45:01.813909, Epoch 602
2018-01-10T04:45:01.813938, open                     0.0975
high                   0.097674
low                     0.09701
close                  0.097112
volume                  924.224
time_utc    2018-01-10 03:56:00
Name: 1515556560, dtype: object
Timestep 2018-01-10 03:56:00 Price 0.097112
2018-01-10T04:45:02.822695, Epoch 603
2018-01-10T04:45:02.822724, open                   0.097199
high                   0.097298
low                     0.09701
close                  0.097011
volume                   513.25
time_utc    2018-01-10 03:57:00
Name: 1515556620, dtype: object

Timestep 2018-01-10 04:19:00 Price 0.097855
2018-01-10T04:45:26.015423, Epoch 626
2018-01-10T04:45:26.015455, open                   0.097993
high                   0.097993
low                    0.097448
close                  0.097775
volume                  461.889
time_utc    2018-01-10 04:20:00
Name: 1515558000, dtype: object
Timestep 2018-01-10 04:20:00 Price 0.097775
2018-01-10T04:45:27.022573, Epoch 627
2018-01-10T04:45:27.022602, open                    0.09765
high                     0.0979
low                    0.097649
close                  0.097815
volume                  303.303
time_utc    2018-01-10 04:21:00
Name: 1515558060, dtype: object
Timestep 2018-01-10 04:21:00 Price 0.097815
2018-01-10T04:45:28.030820, Epoch 628
2018-01-10T04:45:28.030850, open                   0.097815
high                     0.0984
low                    0.097815
close                  0.098014
volume                  507.941
time_utc    2018-01-10 04:22:00
Name: 1515558120, dtype: objec